In [19]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests
import sqlite3
import numpy as np 

In [4]:
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attrib = ["Name", 'MC_USD_Billion']
table_name = 'Largest_banks'
db_name = 'Banks.db'
csv_name = 'Largest_banks_data.csv'


In [13]:
def extract(url, table_attrib):
    page = requests.get(url).text
    data = BeautifulSoup(page,'html.parser')
    df = pd.DataFrame(columns=table_attrib)
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col)!=0:
           if col[1].find('a') is not None:
                data_dict = {"Name": col[1].get_text(strip=True),
                             "MC_USD_Billion": col[2].get_text(strip=True)}
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df,df1], ignore_index=True)
    return df

In [15]:
df = extract(url, table_attrib)
print(df)

                                      Name MC_USD_Billion
0                           JPMorgan Chase         432.92
1                          Bank of America         231.52
2  Industrial and Commercial Bank of China         194.56
3               Agricultural Bank of China         160.68
4                                HDFC Bank         157.91
5                              Wells Fargo         155.87
6                        HSBC Holdings PLC         148.90
7                           Morgan Stanley         140.83
8                  China Construction Bank         139.82
9                            Bank of China         136.81


In [9]:
dataframe = pd.read_csv('exchange_rate (1).csv')
rate_dict = dataframe.set_index('Currency').to_dict()['Rate']

In [10]:
print(rate_dict)

{'GBP': 0.8, 'EUR': 0.93, 'INR': 82.15}


In [61]:
def transform(rate_dict, df): 
     df['MC_GBP_Billion'] = [np.round(float(x)*rate_dict['GBP'],2) for x in df['MC_USD_Billion']]
     df['MC_EUR_Billion'] = [np.round(float(x)*rate_dict['EUR'],2) for x in df['MC_USD_Billion']]
     df['MC_INR_Billion'] = [np.round(float(x)*rate_dict['INR'],2) for x in df['MC_USD_Billion']]
     return df

    


In [62]:
df = transform(rate_dict, df)

In [40]:
df['MC_EUR_Billion']

0    402.62
1    215.31
2    180.94
3    149.43
4    146.86
5    144.96
6    138.48
7    130.97
8    130.03
9    127.23
Name: MC_EUR_Billion, dtype: float64

In [41]:
df

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,346.34,402.62,35564.38
1,Bank of America,231.52,185.22,215.31,19019.37
2,Industrial and Commercial Bank of China,194.56,155.65,180.94,15983.10
3,Agricultural Bank of China,160.68,128.54,149.43,13199.86
4,HDFC Bank,157.91,126.33,146.86,12972.31
5,Wells Fargo,155.87,124.70,144.96,12804.72
6,HSBC Holdings PLC,148.90,119.12,138.48,12232.14
7,Morgan Stanley,140.83,112.66,130.97,11569.18
8,China Construction Bank,139.82,111.86,130.03,11486.21
9,Bank of China,136.81,109.45,127.23,11238.94


In [76]:
def load_to_csv(df, csv_name): 
    df.to_csv(csv_name)

In [82]:
def load_to_db(df, conn, table_name): 
    df.to_sql(table_name, conn, if_exists='replace', index=False)

In [69]:
def run_query(sqlquery, conn): 
    print(sqlquery)
    output = pd.read_sql(sqlquery, conn)
    print(output)

In [70]:
from datetime import datetime
def log_progress(message):
    timestamp = '%y-%h-%d-%H-%M-%S'
    now = datetime.now()
    time = now.strftime(timestamp)
    with open("log_process.txt", "a") as f: 
        f.write(time + ' : ' + message + '\n')
    
    

In [78]:
log_progress('Preliminaries complete. Initiating ETL process')

df = extract(url, table_attrib)

log_progress('Data extraction complete. Initiating Transformation process')

df = transform(rate_dict, df)

log_progress('Data transformation complete. Initiating loading process')




In [79]:
log_progress('Loadinhg into csv ')
load_to_csv(df, csv_name) 
log_progress('csv loaded')

In [83]:
log_progress('loading into database')
conn = sqlite3.connect('Banks.db')
load_to_db(df, conn, table_name)
log_progress('loaded database ')

In [86]:
log_progress('Data loaded to Database as table. Running the query')

sqlquery = f"SELECT * from {table_name} WHERE MC_USD_Billion >= 200"
run_query(sqlquery, conn)

log_progress('Process Complete.')

conn.close()

SELECT * from Largest_banks WHERE MC_USD_Billion >= 200
              Name MC_USD_Billion  MC_GBP_Billion  MC_EUR_Billion  \
0   JPMorgan Chase         432.92          346.34          402.62   
1  Bank of America         231.52          185.22          215.31   

   MC_INR_Billion  
0        35564.38  
1        19019.37  
